In [11]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import heapq

In [12]:
import logging
logging.basicConfig()

In [13]:
all_categories = fetch_20newsgroups().target_names
all_categories

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Возьмём темы из одного раздела, возможно, их будет сложнее отличать друг от друга

In [14]:
categories = [
    'sci.electronics',
    'sci.space',
    'sci.med'
]
train_data = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
test_data = fetch_20newsgroups(subset='test', categories=categories, remove=('headers', 'footers', 'quotes'))

Для векторизации текстов воспользуемся CountVectorizer, он представляет документ как мешок слов. Можно всячески варировать извлечение признаков (убирать редкие слова, убирать частые слова, убирать слова общей лексики, брать биграмы и т.д.)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
CountVectorizer()

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [17]:
count_vectorizer = CountVectorizer(min_df=5, ngram_range=(1, 2)) 

In [18]:
sparse_feature_matrix = count_vectorizer.fit_transform(train_data.data)
sparse_feature_matrix

<1778x10885 sparse matrix of type '<type 'numpy.int64'>'
	with 216486 stored elements in Compressed Sparse Row format>

In [22]:
train_data.data

[u'Another fish to check out is Richard Rast -- he works\nfor Lockheed Missiles, but is on-site at NASA Johnson.\n\nNick Johnson at Kaman Sciences in Colo. Spgs and his\nfriend, Darren McKnight at Kaman in Alexandria, VA.\n\nGood luck.\n\nR. Landis',
 u": As the subject says - Can I use a 4052 for digital signals?  I don't see\n: why it couldn't handle digital signals, but I could be wrong.  Anyone have\n: any advice?  Thanks.\n\nThe switches have a non-negligable on resistance (up to 1k ohm when\npowered by 5 volts) and a maximum current and a Maximum Static\nVoltage Across Switch.  Not a good bet for TTL.  Should work for\nCMOS, but slow things down a bit.  There are 74HC versions that\nhave better specs. but lower max voltage.",
 u'I am looking for current sources for lists of all the home\nmedical tests currently legally available.\nI believe this trend of allowing tests at home where\nfeasible, decreased medical costs by a factor of 10 or\nmore and allows the patient some time and

In [19]:
num_2_words = {
    v: k
    for k, v in count_vectorizer.vocabulary_.iteritems()
}

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

Обучим логистическую регрессию для предсказания темы документа

In [24]:
algo = LogisticRegression()
algo.fit(sparse_feature_matrix, train_data.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Слова с наибольшим положительным весом, являются характерными словами темы

In [25]:
W = algo.coef_.shape[1]
for c in algo.classes_:
    topic_words = [
        num_2_words[w_num]
        for w_num in heapq.nlargest(10, range(W), key=lambda w: algo.coef_[c, w])
    ]
    print ',  '.join(topic_words)


circuit,  electronics,  power,  chips,  parts,  them,  the number,  used,  tv,  ve
msg,  medical,  my,  blood,  disease,  doctor,  health,  treatment,  your,  needles
space,  orbit,  nasa,  thanks for,  launch,  earth,  sorry,  moon,  spacecraft,  solar


Сравним качество на фолдах с качеством на трейне и на отложенном тесте

In [26]:
algo = LogisticRegression()
arr = cross_val_score(algo, sparse_feature_matrix, train_data.target, cv=5, scoring='accuracy')
print arr
print np.mean(arr)

[ 0.8487395   0.84550562  0.83426966  0.83943662  0.82768362]
0.839127002447


Почему это неправильная кроссвалидация?

In [27]:
algo.fit(sparse_feature_matrix, train_data.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
accuracy_score(algo.predict(sparse_feature_matrix), train_data.target)

0.98031496062992129

In [29]:
accuracy_score(algo.predict(count_vectorizer.transform(test_data.data)), test_data.target)

0.79289940828402372

Мы видим переобучение, это проклятие размерности

In [ ]:
algo = LogisticRegression(penalty='l1', C=0.1)
arr = cross_val_score(algo, sparse_feature_matrix, train_data.target, cv=5, scoring='accuracy')
print arr
print np.mean(arr)

In [ ]:
algo.fit(sparse_feature_matrix, train_data.target)

In [ ]:
accuracy_score(algo.predict(sparse_feature_matrix), train_data.target)

In [ ]:
accuracy_score(algo.predict(count_vectorizer.transform(test_data.data)), test_data.target)

Добавление регуляризатора уменьшает отличие на трейне и тесте, но ухудшает качество. Поиграйтесь с параметрами регуляризации, чтобы получить максимальное качество.

Чтобы не делать векторизацию и обучение раздельно, есть удобный класс Pipeline. Он позволяет объединить в цепочку последовательность действий

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([("vectorizer", CountVectorizer(min_df=5, ngram_range=(1, 2))), ("algo", LogisticRegression())])

In [ ]:
pipeline.fit(train_data.data, train_data.target)

In [ ]:
accuracy_score(pipeline.predict(train_data.data), train_data.target)

In [ ]:
accuracy_score(pipeline.predict(test_data.data), test_data.target)

Значения ровно такие же как мы получали ранее, делаяя шаги раздельно.

In [ ]:
from sklearn.pipeline import make_pipeline

При кроссвалидации нужно, чтобы CountVectorizer не обучался на тесте (иначе объекты становятся зависимыми). Pipeline позволяет это просто сделать.

In [ ]:
pipeline = make_pipeline(CountVectorizer(min_df=5, ngram_range=(1, 2)), LogisticRegression())
arr = cross_val_score(pipeline, train_data.data, train_data.target, cv=5, scoring='accuracy')
print arr
print np.mean(arr)

В Pipeline можно добавлять новые шаги препроцессинга данных

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
pipeline = make_pipeline(CountVectorizer(min_df=5, ngram_range=(1, 2)), TfidfTransformer(), LogisticRegression())
arr = cross_val_score(pipeline, train_data.data, train_data.target, cv=5, scoring='accuracy')
print arr
print np.mean(arr)

In [ ]:
pipeline.fit(train_data.data, train_data.target)

In [ ]:
accuracy_score(pipeline.predict(train_data.data), train_data.target)

In [ ]:
accuracy_score(pipeline.predict(test_data.data), test_data.target)

Качество стало немного лучше

#Задание

1. Поиграйтесь с параметрами регуляризации, параметрами CountVectorizer и TfidfTransformer, чтобы получить максимальное качество.
2. Постройте список важных слов и словосочетаний для каждой темы (на основе значений коэффициентов)